In [1]:
import sys
sys.path.append('..')

In [2]:
from minbpe.gpt4 import GPT4Tokenizer
from minbpe.basic import BasicTokenizer

tokenizer = BasicTokenizer()
tokenizer.load(model_file='../output/tokenizer/temp_tokenizer.model')

ValueError: invalid literal for int() with base 10: '<|startoftext|>'